In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import ast
from nltk.corpus import stopwords
from functools import partial
from gensim.corpora import Dictionary
from gensim.models import LdaModel


In [18]:
df = pd.read_csv(r"D:\tesis_u_sabana\data\db_read_clean_inicial\secop_2_adiciones_1_clean.csv", sep="~", encoding='utf-8')

In [19]:
df.head()

identificador         id_contrato             tipo_mins  \
0   co1.ctrmod.499720   co1.pccntr.285227  modificacion general   
1  co1.ctrmod.5061177  co1.pccntr.1422827   adicion en el valor   
2  co1.ctrmod.5570045  co1.pccntr.2134579  modificacion general   
3  co1.ctrmod.1762616   co1.pccntr.812762   adicion en el valor   
4  co1.ctrmod.5593802  co1.pccntr.1676523  modificacion general   

                                         descripcion fecharegistro  \
0  adicionar el valor del contrato de prestacion ...    2018-09-01   
1  c.que de acuerdo al oficio del 26 de octubre d...    2020-01-01   
2  considerando: 1) que la subred integrada de se...    2021-02-01   
3  modificacion  adicion y prorroga nro. 1 al co...    2019-01-01   
4  la subred integrada de servicios de salud sur...    2021-02-01   

                                  descripcion_limpio      mes  
0  ['adicionar', 'valor', 'profesionales', 'n', '...  2018-09  
1  ['c', 'oficio', 'octubre', 'profesional', 'uni...  2020-01  
2  ['considerando', 'subred', 'integrada', 'salud...  2021-02  
3  ['adicion', 'prorroga', 'nro', 'profesionales'...  2019-01  
4  ['subred', 'integrada', 'salud', 'sur', 'occid...  2021-02

In [20]:
# Convert from strings to lists
df['descripcion_limpio'] = df['descripcion_limpio'].apply(ast.literal_eval)

In [21]:
def checkif_stopword(word, stopwords):
    return word not in stopwords

def filter_stopwords(tokens, stopwords):
    return list(filter(partial(checkif_stopword, stopwords=stopwords), tokens))

In [22]:
#añadir mas stopwords
spanish_stopwords = set(stopwords.words('spanish'))
english_stopwords = set(stopwords.words('english'))
other_stopwords = set(["he" , "mas", "si","ser","aunque","hace","tan","solo",
                       "tambien","vez","parece","va","da","hacer","asi","haber",
                       "pues","ver","etc","contrato","servicios","prestacion",
                       "mediante","cuenta","fecha","presente","acuerdo","necesario","contratista",
                        "mil","servicio","requiere","contratos", "supervisor","cumplimiento","modificar",
                        "dar","realizar","objeto","contractual","pesos","modificacion",
                        "conforme","contratacion","gestion","entidad","apoyo","ejecucion",
                        "vigente","normatividad","acta",'integral','area','programas','programa',
                        'proyectos', 'proyecto','direccion','general','nacional','departamento',    
                        'denominado','sistema','oficina','implementacion', 'publica','municipio',
                        'secretaria', 'local','fortalecimiento', 'apoyar','centro',"ps", "pro", "adi", "co", 
                        "corrección", "pccntr", "corregir", "corrige", "actos", "administrativo",
                        "descripcion","contratante","cte","conformidad" ,"realiza" ,"atencion" ,"previo" ,"ajusta",
                        "documentos" ,"modifica" ,"anexos","adjuntos" ,"aporte","actividades" ,"interadministrativo",
                        "atencion","previo","ajusta"
                        ])
stopword_set = spanish_stopwords.union(english_stopwords,other_stopwords)

df['descripcion_limpio'] = df['descripcion_limpio'].apply(partial(filter_stopwords, stopwords=stopword_set))

In [23]:
texts = df['descripcion_limpio'].tolist()  # list of token lists
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [24]:
lda_gensim = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    passes=5,             # passes over corpus
    iterations=50,
    chunksize=6000,
    eval_every=None,
    random_state=7
)



In [25]:
def get_dominant_topic(lda_model, bow):
    topic_probs = lda_model.get_document_topics(bow)
    return max(topic_probs, key=lambda x: x[1])[0] if topic_probs else None

df['dominant_topic'] = [get_dominant_topic(lda_gensim, doc) for doc in corpus]

In [26]:
from gensim.models import CoherenceModel

coherence_model = CoherenceModel(
    model=lda_gensim,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()
print(f"🧠 Coherence Score (c_v): {coherence_score:.4f}")

🧠 Coherence Score (c_v): 0.7265


In [27]:
for idx, topic in lda_gensim.print_topics(num_words=10):
    print(f"🔹 Topic #{idx + 1}: {topic}")

🔹 Topic #1: 0.196*"prorroga" + 0.177*"adicion" + 0.061*"solicitud" + 0.031*"valor" + 0.024*"adiciona" + 0.022*"justificacion" + 0.020*"tiempo" + 0.018*"otrosi" + 0.017*"adicional" + 0.014*"supervision"
🔹 Topic #2: 0.012*"salud" + 0.007*"garantizar" + 0.007*"fin" + 0.007*"suministro" + 0.006*"cuyo" + 0.006*"anterior" + 0.005*"debido" + 0.005*"diferentes" + 0.005*"cuales" + 0.005*"adicion"
🔹 Topic #3: 0.043*"modificatorio" + 0.036*"suspension" + 0.030*"convenio" + 0.027*"terminacion" + 0.027*"reinicio" + 0.024*"solicitud" + 0.021*"documento" + 0.018*"anexo" + 0.014*"otrosi" + 0.013*"condiciones"
🔹 Topic #4: 0.115*"n" + 0.019*"celebrado" + 0.019*"icbf" + 0.016*"recursos" + 0.016*"primera" + 0.015*"regional" + 0.014*"instituto" + 0.014*"memorando" + 0.011*"grupo" + 0.011*"infancia"
🔹 Topic #5: 0.025*"subred" + 0.024*"salud" + 0.024*"sur" + 0.024*"occidente" + 0.024*"integrada" + 0.023*"requerimiento" + 0.022*"derecho" + 0.022*"administrativa" + 0.022*"asistencial" + 0.022*"consideraciones"

In [28]:
df.head(10)

identificador         id_contrato             tipo_mins  \
0   co1.ctrmod.499720   co1.pccntr.285227  modificacion general   
1  co1.ctrmod.5061177  co1.pccntr.1422827   adicion en el valor   
2  co1.ctrmod.5570045  co1.pccntr.2134579  modificacion general   
3  co1.ctrmod.1762616   co1.pccntr.812762   adicion en el valor   
4  co1.ctrmod.5593802  co1.pccntr.1676523  modificacion general   
5   co1.ctrmod.853424   co1.pccntr.718649  modificacion general   
6  co1.ctrmod.5218272  co1.pccntr.1799276  modificacion general   
7  co1.ctrmod.2023804   co1.pccntr.799322  modificacion general   
8   co1.ctrmod.291127   co1.pccntr.229207            suspension   
9  co1.ctrmod.4896636  co1.pccntr.1263192  modificacion general   

                                         descripcion fecharegistro  \
0  adicionar el valor del contrato de prestacion ...    2018-09-01   
1  c.que de acuerdo al oficio del 26 de octubre d...    2020-01-01   
2  considerando: 1) que la subred integrada de se...    2021-02-01   
3  modificacion  adicion y prorroga nro. 1 al co...    2019-01-01   
4  la subred integrada de servicios de salud sur...    2021-02-01   
5  teniendo en cuenta el concepto emitido por la ...    2019-03-06   
6  contrato 1799276 de 2020; suscrito entre el de...    2020-01-01   
7  se requiere modificar el presente contrato par...    2020-01-01   
8  reanudacion del contrato 498disan-arc-sp-hona...    2018-05-02   
9  que el icbf  regional norte de santander; con...    2020-01-02   

                                  descripcion_limpio      mes  dominant_topic  
0  [adicionar, valor, profesionales, n, suma, qui...  2018-09               8  
1  [c, oficio, octubre, profesional, universitari...  2020-01               1  
2  [considerando, subred, integrada, salud, sur, ...  2021-02               4  
3  [adicion, prorroga, nro, profesionales, celebr...  2019-01               1  
4  [subred, integrada, salud, sur, occidente, nut...  2021-02               1  
5  [concepto, emitido, subdireccion, financiera, ...  2019-03               2  
6  [suscrito, estadistica, dane, miguel, fernando...  2020-01               6  
7  [mision, vision, igual, necesidad, profesional...  2020-01               4  
8  [reanudacion, disan, arc, sp, honac, terminaci...  2018-05               2  
9  [icbf, regional, norte, santander, proposito, ...  2020-01               9

In [29]:
# Contar cantidad de filas por cada tema dominante
conteo_temas = df['dominant_topic'].value_counts().sort_index()

# Mostrar el resultado
print("🧾 Cantidad de filas por tema dominante:")
print(conteo_temas)


🧾 Cantidad de filas por tema dominante:
dominant_topic
0    1515567
1     155152
2     238266
3     124994
4     364333
5     337523
6      52698
7      99622
8     298147
9     166415
Name: count, dtype: int64


In [34]:
df['riesgo_contractual'] = df['dominant_topic'].apply(lambda x: x in {0,1, 2, 8,9, 10})

In [35]:
df.head(9)

identificador         id_contrato             tipo_mins  \
0   co1.ctrmod.499720   co1.pccntr.285227  modificacion general   
1  co1.ctrmod.5061177  co1.pccntr.1422827   adicion en el valor   
2  co1.ctrmod.5570045  co1.pccntr.2134579  modificacion general   
3  co1.ctrmod.1762616   co1.pccntr.812762   adicion en el valor   
4  co1.ctrmod.5593802  co1.pccntr.1676523  modificacion general   
5   co1.ctrmod.853424   co1.pccntr.718649  modificacion general   
6  co1.ctrmod.5218272  co1.pccntr.1799276  modificacion general   
7  co1.ctrmod.2023804   co1.pccntr.799322  modificacion general   
8   co1.ctrmod.291127   co1.pccntr.229207            suspension   

                                         descripcion fecharegistro  \
0  adicionar el valor del contrato de prestacion ...    2018-09-01   
1  c.que de acuerdo al oficio del 26 de octubre d...    2020-01-01   
2  considerando: 1) que la subred integrada de se...    2021-02-01   
3  modificacion  adicion y prorroga nro. 1 al co...    2019-01-01   
4  la subred integrada de servicios de salud sur...    2021-02-01   
5  teniendo en cuenta el concepto emitido por la ...    2019-03-06   
6  contrato 1799276 de 2020; suscrito entre el de...    2020-01-01   
7  se requiere modificar el presente contrato par...    2020-01-01   
8  reanudacion del contrato 498disan-arc-sp-hona...    2018-05-02   

                                  descripcion_limpio      mes  dominant_topic  \
0  [adicionar, valor, profesionales, n, suma, qui...  2018-09               8   
1  [c, oficio, octubre, profesional, universitari...  2020-01               1   
2  [considerando, subred, integrada, salud, sur, ...  2021-02               4   
3  [adicion, prorroga, nro, profesionales, celebr...  2019-01               1   
4  [subred, integrada, salud, sur, occidente, nut...  2021-02               1   
5  [concepto, emitido, subdireccion, financiera, ...  2019-03               2   
6  [suscrito, estadistica, dane, miguel, fernando...  2020-01               6   
7  [mision, vision, igual, necesidad, profesional...  2020-01               4   
8  [reanudacion, disan, arc, sp, honac, terminaci...  2018-05               2   

   riesgo_contractual  
0                True  
1                True  
2               False  
3                True  
4                True  
5                True  
6               False  
7               False  
8                True

In [37]:
df.to_csv(r"D:\tesis_u_sabana\data\db_1\adiciones_1_topic_modeling.csv", sep="~", encoding='utf-8', index=False)